# CarbonPlan Project Reports API Demo

This notebook demonstrates how to use CarbonPlan's API to access and analyze project reports.

In [1]:
import requests
import pandas as pd
import altair as alt

from IPython.display import JSON

### Schema

All API access is over HTTPS, and accessed from https://api.carbonplan.org. All data is sent and received as JSON. The full schema is available via the API: 

In [2]:
schema = requests.get('https://api.carbonplan.org/schema.json').json()

# list objects defined in CarbonPlan's schema
JSON(schema)

<IPython.core.display.JSON object>

In [3]:
# or get the full schema for a specific object
schema = requests.get('https://api.carbonplan.org/schema/ProjectCollection.json').json()
JSON(schema)

<IPython.core.display.JSON object>

### Projects

CarbonPlan's full ProjectCollection is available at https://api.carbonplan.org/projects.json:

In [4]:
project_collection = requests.get('https://api.carbonplan.org/projects.json').json()
JSON(project_collection, root='project_collection')

<IPython.core.display.JSON object>

Individual projects can be requested using query parameters. In the example below, we point our API request at `id=STRP01`:

In [5]:
one_project = requests.get('https://api.carbonplan.org/projects.json?id=STRP01').json()
JSON(one_project)

<IPython.core.display.JSON object>

The Projects data is also available in CSV format: https://api.carbonplan.org/projects.csv

In [6]:
projects = pd.read_csv('https://api.carbonplan.org/projects.csv', index_col='name')
projects.head()

,id,description,"('mechanism', 'value')","('mechanism', 'units')","('mechanism', 'rating')","('mechanism', 'notes')","('mechanism', 'comment')","('mechanism', 'removal')","('mechanism', 'avoided')","('volume', 'value')",...,"('specificity', 'value')","('specificity', 'units')","('specificity', 'rating')","('specificity', 'notes')","('specificity', 'comment')","('source', 'name')","('source', 'license')","('source', 'url')","('tag', 0)","('tag', 1)"
name,,,,,,,,,,,,,,,,,,,,,
Climeworks,STRP001,Direct air capture and mineralization.,NaN,NaN,1.0,Removes carbon from the atmosphere using a sol...,NaN,1.0,0.0,50.0,...,1.0,NaN,NaN,NaN,Basic technology is well understood through bo...,Stripe 2020 Negative Emissions Purchase,CC-BY-4.0,https://github.com/stripe/negative-emissions-s...,dac,mineralization
CarbonCure,STRP002,Carbon reductions in concrete.,NaN,NaN,1.0,Captured industrial CO2 source is mineralized ...,"From a narrow lifecycle perspective, project o...",0.0,1.0,60000.0,...,1.0,NaN,NaN,NaN,Publications on specific techology are not yet...,Stripe 2020 Negative Emissions Purchase,CC-BY-4.0,https://github.com/stripe/negative-emissions-s...,mineralization,materials
Nori Harborview Farms,STRP003,Increasing soil carbon sequestration and soil ...,NaN,NaN,0.0,Broker facilitates atmospheric carbon removal ...,Multiple established soil carbon removal pathw...,1.0,0.0,14011.0,...,1.0,NaN,NaN,NaN,Several project details (e.g. plot locations) ...,Stripe 2020 Negative Emissions Purchase,CC-BY-4.0,https://github.com/stripe/negative-emissions-s...,soil,broker
Indigo Ag,STRP004,Regenerative farming practices for soil carbon...,NaN,NaN,0.0,Broker facilitates atmospheric carbon removal ...,Multiple established soil carbon removal pathw...,1.0,0.0,550000.0,...,0.0,NaN,NaN,NaN,No information or data on specific projects ar...,Stripe 2020 Negative Emissions Purchase,CC-BY-4.0,https://github.com/stripe/negative-emissions-s...,soil,broker
Biorecro,STRP005,Supporting bio-energy with carbon capture and ...,NaN,NaN,0.0,"Broker facilitates BECCS projects, which avoid...",BECCS is a well-established technology. Projec...,1.0,1.0,50000.0,...,0.0,NaN,NaN,NaN,Basic technology is well understood. Few data ...,Stripe 2020 Negative Emissions Purchase,CC-BY-4.0,https://github.com/stripe/negative-emissions-s...,biomass,broker


### Sample Analysis

Now that we've gone over how to access CarbonPlan's API, we can turn to some sample analysis. Here we'll simply take the full project collection as read from the CSV endpoint above.

In [7]:
metrics = ['cost', 'volume', 'negativity', 'permanence']
metrics_df = pd.DataFrame.from_dict({m: projects[f"('{m}', 'value')"] for m in metrics})
metrics_df['tag'] = projects["('tag', 0)"]
metrics_df.head()

,cost,volume,negativity,permanence,tag
name,,,,,
Climeworks,767.5,50.0,0.900,1000.0,dac
CarbonCure,100.0,60000.0,0.824,1000.0,mineralization
Nori Harborview Farms,16.5,14011.0,NaN,10.0,soil
Indigo Ag,20.0,550000.0,NaN,100.0,soil
Biorecro,99.0,50000.0,0.900,1000.0,biomass


From here, we can start doing some real analysis and data visualization with our metrics `DataFrame`.

In the section below, we'll make a few of charts that are shown on https://reports.carbonplan.org. We use [Altair](https://altair-viz.github.io/) to make a few interactive vizualizations.

In [8]:
colors = {
    'mineralization': '#a9b4c4',
    'soil': '#ea9755',
    'biomass': '#d4c05e',
    'forests': '#7eb36a',
    'ocean': '#64b9c4',
    'dac': '#bc85d9'
}
color_scale = alt.Color('tag:N', scale=alt.Scale(domain=list(colors.keys()), range=list(colors.values())))
tooltip = ['name', 'volume', 'cost', 'permanence', 'negativity']

In [9]:
# selectors
brush = alt.selection_interval(encodings=['x', 'y'])
click = alt.selection_multi()

# chart
chart = alt.Chart(metrics_df.reset_index()).mark_circle(size=200).encode(
    color=color_scale,
    opacity=alt.condition(brush | click, alt.OpacityValue(1), alt.OpacityValue(0.4)),
    tooltip=tooltip
).properties(
    width=250,
    height=250
).add_selection(
    brush, click
)

# hconcat
(
    chart.encode(
        alt.X('cost', 
              axis=alt.Axis(title='COST $/tCO2', tickCount=3), 
              scale=alt.Scale(type='log', domain=[2, 2000], nice=False)),
        alt.Y('tag', axis=alt.Axis(title=''))
    ) |
    chart.encode(
        alt.X('volume', 
              axis=alt.Axis(title='VOLUME tCO2', tickCount=3), 
              scale=alt.Scale(type='log', domain=[2, 100000000])),
        alt.Y('tag', axis=alt.Axis(title=''))
    ) |
    chart.encode(
        alt.X('negativity', 
              axis=alt.Axis(title='NEGATIVITY', tickCount=2), 
              scale=alt.Scale(domain=[-0.1, 1.1])),
        alt.Y('tag', axis=alt.Axis(title=''))
    ) |
    chart.encode(
        alt.X('permanence', 
              axis=alt.Axis(title='PERMANENCE years', tickCount=3), 
              scale=alt.Scale(type='log', domain=[0.6, 2000], nice=False)),
        alt.Y('tag', axis=alt.Axis(title='')))
).configure_axis(grid=False).resolve_scale(
    y='shared'
)

alt.HConcatChart(...)

We can also explore interactions among variables:

In [10]:
# chart
chart = alt.Chart(metrics_df.reset_index()).mark_circle(size=200).encode(
    color=color_scale,
    opacity=alt.condition(brush | click, alt.OpacityValue(1), alt.OpacityValue(0.4)),
    tooltip=tooltip
).properties(
    width=250,
    height=250
).add_selection(
    brush, click
)

# hconcat
(
    chart.encode(
        alt.X('volume', 
              axis=alt.Axis(title='VOLUME tCO2', tickCount=3), 
              scale=alt.Scale(type='log', domain=[2, 100000000])),
        alt.Y('cost', 
              axis=alt.Axis(title='COST $/tCO2', tickCount=3), 
              scale=alt.Scale(type='log', domain=[2, 2000], nice=False)),
    ) |
    chart.encode(
        alt.X('permanence', 
              axis=alt.Axis(title='PERMANENCE years', tickCount=3), 
              scale=alt.Scale(type='log', domain=[0.6, 2000], nice=False)),
        alt.Y('cost', 
              axis=alt.Axis(title='COST $/tCO2', tickCount=3), 
              scale=alt.Scale(type='log', domain=[2, 2000], nice=False)))
).configure_axis(grid=False).resolve_scale(
    y='shared'
)

alt.HConcatChart(...)